# (1) Read data
pip install jieba

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#設定matplot可以直接在Notebook中畫出，不需要執行plt.plot()
%matplotlib inline 

# 設定 Seaborn 的顯示樣式
# https://seaborn.pydata.org/generated/seaborn.set_style.html
sns.set_style('whitegrid')

#載入資料
#data = pd.read_csv("data/Op_Dcard_markeup.csv")
data = pd.read_csv("data/Op_Dcard_markeup_w.csv")
print(data.shape)  #type(pandas.core.frame.DataFrame)
data.head(3)

(30027, 9)


,id,title,content,excerpt,createdAt,updatedAt,forumName,forumAlias,gender
0,235081118,#討論 注意詐騙🔺林三益官網疑似個資外洩🔺,（不知道放這個分類對不對，有需要我再改）\n\n這幾天一直收到林三益的簡訊如圖\nhttps...,（不知道放這個分類對不對，有需要我再改），這幾天一直收到林三益的簡訊如圖，本來不以為意的，結...,2020-12-30T11:25:39.607Z,2020-12-30T11:25:39.607Z,美妝,makeup,F
1,235080929,請問留言中的女孩們是否有好轉呢,此篇文章為轉貼文章，請更新至最新版本觀看完整內容。,NaN,2020-12-30T10:48:14.384Z,2020-12-30T10:48:14.384Z,美妝,makeup,M
2,235080758,#分享 去角質DIY feat妮維雅小藍罐,因為看了婊姐的小藍罐去角質廣告\n想說材料我都有就照著試了幾次，效果真的不錯\n只是我第一次...,因為看了婊姐的小藍罐去角質廣告，想說材料我都有就照著試了幾次，效果真的不錯，只是我第一次咖啡...,2020-12-30T10:12:29.843Z,2020-12-30T10:12:29.843Z,美妝,makeup,F


# (2) 格式轉換

- transform Type(pandas.core.series.Series) to list
- transform Type(list) to str
- addition: https://blog.csdn.net/Zx_whu/article/details/61926655

In [11]:
dt_list = data['content'].tolist()   # transform Type(pandas.core.series.Series) to list
dt_list[0]      #Content 第一篇文章
#dt_list[1]     #Content 第二篇文章
#dt_list[87]    #最後一篇文章 #type(str)

'（不知道放這個分類對不對，有需要我再改）\n\n這幾天一直收到林三益的簡訊如圖\nhttps://i.imgur.com/vixShPl.png\n本來不以為意的\n\n結果剛剛接到一通 Whoscall 也沒辨認出是什麼的市話\n（詐騙號碼是我回報之後寫上去的，打來時沒有顯示）\nhttps://i.imgur.com/Xr5S2MF.png\n\n操著外地口音的婦女\n詢問我先前是否有購買某項商品\n（但那已經是九月購買的了，不是最新的購買紀錄）\n我回答有之後\n她說\n\n「最近系統被駭了\n你被自動升級成高級會員\n每個月會自動扣款一萬二\n請問您有意願成為我們的高級會員嗎？\n還是要幫忙解除？\n我們想瞭解你的意願」\n\n我只回了\n「我瞭解到你們是詐騙了 ^_^」\n\n感覺林三益的官網出事了\n之前有在林三益官網購買過的女孩請小心\n接到這種電話\n要不直接掛\n要不打165求證\n\n我是很想看看他們要怎麼扣我一萬二啦 ^_^\n大家接到這種詐騙電話都是怎麼處理的呢？'

In [12]:
dt_str= ''.join(map(str,dt_list))
#dt_str  #type(str)

# for i in dt_list:
#     str2 = ''
#     print(str2.join(i))
#     dt_str = i.join(i)

# (3) Jeiba 語句分析

- https://github.com/ldkrsi/jieba-zh_TW
- https://blog.kennycoder.io/2020/02/12/Python-知名Jieba中文斷詞工具教學/

jieba.cut生成的是一个生成器，generator，也就是可以通过for循环来取里面的每一个词。
- 全模式: 把句子中所有的可以成詞的詞語都斷出来
- jieba.cut(sentence, cut_all=True)
- 精確模式 : 將句子最精確的切開，適合文本分析
- jieba.cut(sentence, cut_all=False)
- 搜索引擎模式: 在精確模式的基礎上，對長的詞語再次切分，提高召回率
- jieba.cut_for_search(sentence)

jieba.lcut 直接生成的就是一个list
- jieba.lcut(news)

In [13]:
import jieba
import jieba.analyse

jieba.load_userdict('data/usedict.txt')   #adjust
#jieba.add_word("")                       #adjust (quick)

news = dt_str
seg_list = jieba.lcut(news)
print(seg_list)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
tags = jieba.analyse.extract_tags(news, topK=50, withWeight=True)
for tag in tags:
    print('tf-idf:', round(tag[1],3),'| word:', tag[0])

tf-idf: 0.279 | word: https
tf-idf: 0.215 | word: com
tf-idf: 0.212 | word: imgur
tf-idf: 0.202 | word: jpg
tf-idf: 0.087 | word: dcard
tf-idf: 0.086 | word: tw
tf-idf: 0.076 | word: megapx
tf-idf: 0.076 | word: images
tf-idf: 0.075 | word: assets
tf-idf: 0.075 | word: 覺得
tf-idf: 0.075 | word: jpeg
tf-idf: 0.062 | word: 比較
tf-idf: 0.06 | word: 沒有
tf-idf: 0.057 | word: 因為
tf-idf: 0.05 | word: 不會
tf-idf: 0.047 | word: 顏色
tf-idf: 0.044 | word: 還是
tf-idf: 0.042 | word: 640
tf-idf: 0.04 | word: 保濕
tf-idf: 0.04 | word: 感覺
tf-idf: 0.039 | word: 喜歡
tf-idf: 0.038 | word: 真的
tf-idf: 0.033 | word: 有點
tf-idf: 0.032 | word: 一點
tf-idf: 0.03 | word: 大家
tf-idf: 0.03 | word: 時候
tf-idf: 0.029 | word: 皮膚
tf-idf: 0.028 | word: 一樣
tf-idf: 0.027 | word: 產品
tf-idf: 0.026 | word: 分享
tf-idf: 0.026 | word: 適合
tf-idf: 0.026 | word: 可以
tf-idf: 0.025 | word: 什麼
tf-idf: 0.025 | word: 推薦
tf-idf: 0.025 | word: 開始
tf-idf: 0.024 | word: 然後
tf-idf: 0.024 | word: 質地
tf-idf: 0.024 | word: 唇膏
tf-idf: 0.023 | word: 精華
tf-id

In [ ]:
import jieba
import jieba.posseg as pseg
words = pseg.cut(dt)
for word, flag in words:
    print('%s %s' % (word, flag))

### 詞性分析

In [7]:
seg_list

['這週',
 '就是',
 '🎄',
 '聖誕節',
 '惹',
 '⛄',
 '️',
 '\n',
 '各',
 '位',
 '卡友',
 '們',
 '餐廳',
 '跟',
 '交換',
 '禮物',
 '都',
 '準備好',
 '了',
 '！',
 '\n',
 '現在',
 '應該',
 '是',
 '萬事俱備',
 ' ',
 '只',
 '欠',
 '東風',
 '\n',
 '是',
 '聖誕',
 '妝容',
 '啦',
 '！',
 '！',
 '😍',
 '\n',
 '來',
 '～',
 '不用',
 '擔心',
 '\n',
 '小',
 '天使',
 '幫',
 '各',
 '位',
 '整理',
 '了',
 '卡友',
 '們',
 '分享',
 '的',
 '聖誕',
 '妝容',
 '\n',
 '這',
 '篇',
 '真的',
 '要',
 '好好',
 '閱讀',
 ' ',
 ' ',
 '當天',
 '亮麗',
 '登場',
 '！',
 '💖',
 '\n',
 'https',
 ':',
 '/',
 '/',
 'i',
 '.',
 'imgur',
 '.',
 'com',
 '/',
 'JGqeHb2',
 '.',
 'jpg',
 '\n',
 '這邊',
 '也',
 '給',
 '大家',
 '一',
 '個',
 'Tip',
 '\n',
 '聖誕',
 '妝容',
 '的',
 '精髓',
 '在',
 ' ',
 'bling',
 ' ',
 'bling',
 '✨',
 '\n',
 '眼妝',
 '就',
 '給',
 '他',
 '閃到',
 '比',
 '聖誕樹',
 '還',
 '閃',
 '就',
 '對',
 '了',
 '\n',
 '幫',
 '各',
 '位',
 '找',
 '幾',
 '個',
 '範例',
 '～',
 '每',
 '個',
 '眼妝',
 '都',
 '是',
 '亮晶晶',
 '！',
 '\n',
 'https',
 ':',
 '/',
 '/',
 'www',
 '.',
 'dcard',
 '.',
 'tw',
 '/',
 'f',
 '/',
 'makeup',
 '/',
 'p',
 '/',
 '234